In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
pd.set_option('display.max_row',None)
pd.set_option('display.expand_frame_repr',False)
pd.set_option('display.max_colwidth',None)

In [2]:
deliveries1 = pd.read_csv('deliveries1.csv')
matches1 = pd.read_csv('matches1.csv')

In [3]:
df=deliveries1

In [4]:
mdf=matches1

In [5]:
#matches.rename(columns = {'id':'match_id'}, inplace = True)

In [6]:
mdf.columns

Index(['id', 'season', 'city', 'date', 'match_type', 'player_of_match',
       'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner',
       'result', 'result_margin', 'target_runs', 'target_overs', 'super_over',
       'method', 'umpire1', 'umpire2'],
      dtype='object')

In [7]:
# Merge the DataFrames using the correct columns
comb = pd.merge(df, mdf, left_on='match_id', right_on='id', how='left')


In [8]:
# Check the columns in both DataFrames
print(comb.columns)



Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'id', 'season', 'city', 'date',
       'match_type', 'player_of_match', 'venue', 'team1', 'team2',
       'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin',
       'target_runs', 'target_overs', 'super_over', 'method', 'umpire1',
       'umpire2'],
      dtype='object')


In [9]:
def wintoss_winmatch(toss_winner,winner):
    if toss_winner == winner:
        return 1
    else:
        return 0

In [10]:
mdf['wintoss_winmatch'] = mdf.apply(lambda x: wintoss_winmatch(x['toss_winner'],x['winner']),axis = 1)

In [11]:
mdf[['winner','toss_winner','wintoss_winmatch']].head(10)

winner                  toss_winner  wintoss_winmatch
0        Kolkata Knight Riders  Royal Challengers Bangalore                 0
1          Chennai Super Kings          Chennai Super Kings                 1
2             Delhi Daredevils             Rajasthan Royals                 0
3  Royal Challengers Bangalore               Mumbai Indians                 0
4        Kolkata Knight Riders              Deccan Chargers                 0
5             Rajasthan Royals              Kings XI Punjab                 0
6             Delhi Daredevils              Deccan Chargers                 0
7          Chennai Super Kings               Mumbai Indians                 0
8             Rajasthan Royals             Rajasthan Royals                 1
9              Kings XI Punjab               Mumbai Indians                 0

In [12]:
100*len(mdf[mdf.wintoss_winmatch == 1])/len(mdf)

50.593607305936075

In [13]:
len(mdf)

1095

In [14]:
pd.DataFrame(mdf.groupby('season')['wintoss_winmatch'].sum()/mdf.groupby('season')['wintoss_winmatch'].count())

wintoss_winmatch
season                   
2007/08          0.482759
2009             0.578947
2009/10          0.516667
2011             0.520548
2012             0.445946
2013             0.473684
2014             0.500000
2015             0.474576
2016             0.566667
2017             0.576271
2018             0.533333
2019             0.600000
2020/21          0.416667
2021             0.583333
2022             0.486486
2023             0.459459
2024             0.436620

In [15]:
team_name = "Kolkata Knight Riders"
opponent_teams = ["Royal Challengers Bangalore", "Royal Challengers Bengaluru"]  # Check for both names

# Filter matches where KKR played against either name variant of RCB
kkr_rcb_matches = mdf[
    ((mdf['team1'] == team_name) & (mdf['team2'].isin(opponent_teams))) |
    ((mdf['team1'].isin(opponent_teams)) & (mdf['team2'] == team_name))
].copy()

# Ensure the date column is in datetime format
kkr_rcb_matches['date'] = pd.to_datetime(kkr_rcb_matches['date'])

# Identify the opponent team
kkr_rcb_matches['opponent'] = kkr_rcb_matches.apply(lambda x: x['team1'] if x['team2'] == team_name else x['team2'], axis=1)

# Group by opponent, venue, date, toss winner, toss decision, match winner, result margin, and target runs
kkr_rcb_analysis = kkr_rcb_matches.groupby(
    ['opponent', 'venue', 'date', 'toss_winner', 'toss_decision', 'winner', 'result_margin', 'target_runs']
).size().reset_index(name='matches_played')

# Sort by date in ascending order
kkr_rcb_analysis = kkr_rcb_analysis.sort_values(by='date')

# Display the result
(kkr_rcb_analysis)


opponent                                     venue       date                  toss_winner toss_decision                       winner  result_margin  target_runs  matches_played
14  Royal Challengers Bangalore                     M Chinnaswamy Stadium 2008-04-18  Royal Challengers Bangalore         field        Kolkata Knight Riders          140.0        223.0               1
1   Royal Challengers Bangalore                              Eden Gardens 2008-05-08        Kolkata Knight Riders           bat        Kolkata Knight Riders            5.0        130.0               1
13  Royal Challengers Bangalore                                 Kingsmead 2009-04-29        Kolkata Knight Riders           bat  Royal Challengers Bangalore            5.0        140.0               1
30  Royal Challengers Bangalore                           SuperSport Park 2009-05-12  Royal Challengers Bangalore         field  Royal Challengers Bangalore            6.0        174.0               1
2   Royal Challengers Bangalore                              Eden Gardens 2010-03-14        Kolkata Knight Riders         field        Kolkata Knight Riders            7.0        136.0               1
15  Royal Challengers Bangalore                     M Chinnaswamy Stadium 2010-04-10  Royal Challengers Bangalore         field  Royal Challengers Bangalore            7.0        161.0               1
3   Royal Challengers Bangalore                              Eden Gardens 2011-04-22  Royal Challengers Bangalore         field  Royal Challengers Bangalore            9.0        172.0               1
16  Royal Challengers Bangalore                     M Chinnaswamy Stadium 2011-05-14  Royal Challengers Bangalore         field  Royal Challengers Bangalore            4.0        102.0               1
17  Royal Challengers Bangalore                     M Chinnaswamy Stadium 2012-04-10  Royal Challengers Bangalore         field        Kolkata Knight Riders           42.0        166.0               1
4   Royal Challengers Bangalore                              Eden Gardens 2012-04-28        Kolkata Knight Riders           bat        Kolkata Knight Riders           47.0        191.0               1
18  Royal Challengers Bangalore                     M Chinnaswamy Stadium 2013-04-11  Royal Challengers Bangalore         field  Royal Challengers Bangalore            8.0        155.0               1
12  Royal Challengers Bangalore        JSCA International Stadium Complex 2013-05-12        Kolkata Knight Riders         field        Kolkata Knight Riders            5.0        116.0               1
26  Royal Challengers Bangalore                   Sharjah Cricket Stadium 2014-04-24  Royal Challengers Bangalore         field        Kolkata Knight Riders            2.0        151.0               1
5   Royal Challengers Bangalore                              Eden Gardens 2014-05-22  Royal Challengers Bangalore         field        Kolkata Knight Riders           30.0        196.0               1
6   Royal Challengers Bangalore                              Eden Gardens 2015-04-11  Royal Challengers Bangalore         field  Royal Challengers Bangalore            3.0        178.0               1
19  Royal Challengers Bangalore                     M Chinnaswamy Stadium 2015-05-02  Royal Challengers Bangalore         field  Royal Challengers Bangalore            7.0        112.0               1
20  Royal Challengers Bangalore                     M Chinnaswamy Stadium 2016-05-02        Kolkata Knight Riders         field        Kolkata Knight Riders            5.0        186.0               1
7   Royal Challengers Bangalore                              Eden Gardens 2016-05-16  Royal Challengers Bangalore         field  Royal Challengers Bangalore            9.0        184.0               1
8   Royal Challengers Bangalore                              Eden Gardens 2017-04-23  Royal Challengers Bangalore         field        Kolkata Knight Riders           82.0        132.0               1
21 

In [38]:
import pandas as pd

# Input Variables
team_name = "Kolkata Knight Riders"
unified_opponent_name = "Royal Challengers Bangalore (RCB)"
opponent_teams = ["Royal Challengers Bangalore", "Royal Challengers Bengaluru"]
home_venues = ["Eden Gardens", "Eden Gardens, Kolkata"]

# Standardizing Opponent Team Names
comb['team1'] = comb['team1'].replace(opponent_teams, unified_opponent_name)
comb['team2'] = comb['team2'].replace(opponent_teams, unified_opponent_name)
comb['toss_winner'] = comb['toss_winner'].replace(opponent_teams, unified_opponent_name)
comb['winner'] = comb['winner'].replace(opponent_teams, unified_opponent_name)

# Filtering Matches Involving the Two Teams
filtered_matches = comb[
    ((comb['team1'] == team_name) & (comb['team2'] == unified_opponent_name)) |
    ((comb['team1'] == unified_opponent_name) & (comb['team2'] == team_name))
].copy()

# Remove duplicates based on match_id
filtered_matches = filtered_matches.drop_duplicates(subset='match_id')

# Ensure Date Column is in Correct Format
filtered_matches['date'] = pd.to_datetime(filtered_matches['date'])

# Opponent Column for Clarity
filtered_matches['opponent'] = filtered_matches.apply(
    lambda x: x['team1'] if x['team2'] == team_name else x['team2'], axis=1
)

# Counting Total Matches Played & Wins
total_matches = filtered_matches.shape[0]
team_wins = filtered_matches['winner'].value_counts()

# Head-to-Head at KKR Home Venues
home_matches = filtered_matches[filtered_matches['venue'].isin(home_venues)]
total_home_matches = home_matches.shape[0]
home_wins = home_matches['winner'].value_counts()
kkr_home_wins = home_wins.get(team_name, 0)
rcb_home_wins = home_wins.get(unified_opponent_name, 0)

# Last 5 Head-to-Head Matches
top_5_matches = filtered_matches.sort_values(by='date', ascending=False).head(5)
top_5_matches = top_5_matches[['date', 'team1', 'team2', 'winner', 'result_margin']]

# Toss Analysis
toss_win_count = filtered_matches['toss_winner'].value_counts()
toss_decision_correct = filtered_matches[filtered_matches['toss_winner'] == filtered_matches['winner']]['toss_winner'].value_counts()
toss_decision_wrong = filtered_matches[filtered_matches['toss_winner'] != filtered_matches['winner']]['toss_winner'].value_counts()

# Home & Away Analysis
away_matches = filtered_matches[~filtered_matches['venue'].isin(home_venues)]

home_toss_wins = home_matches[home_matches['toss_winner'] == team_name].shape[0]
home_match_wins = home_matches[home_matches['winner'] == team_name].shape[0]

away_toss_wins = away_matches[away_matches['toss_winner'] == team_name].shape[0]
away_match_wins = away_matches[away_matches['winner'] == team_name].shape[0]

# Calculate Win Percentages Safely
home_toss_win_percentage = (home_toss_wins / total_home_matches) * 100 if total_home_matches > 0 else 0
home_match_win_percentage = (home_match_wins / total_home_matches) * 100 if total_home_matches > 0 else 0
away_toss_win_percentage = (away_toss_wins / away_matches.shape[0]) * 100 if away_matches.shape[0] > 0 else 0
away_match_win_percentage = (away_match_wins / away_matches.shape[0]) * 100 if away_matches.shape[0] > 0 else 0

# Toss Decision Analysis
toss_decision_counts = filtered_matches.groupby(['toss_winner', 'toss_decision']).size().unstack(fill_value=0)
toss_decision_results = filtered_matches.groupby(['toss_winner', 'toss_decision', 'winner']).size().unstack(fill_value=0)

# Printing Results
print(f"\n🏏 **Total Head-to-Head Matches Between {team_name} & {unified_opponent_name}:** {total_matches}")
print("\n🏆 **Head-to-Head Wins:**")
print(team_wins)

print(f"\n🏟 **Head-to-Head at {home_venues[0]} (KKR Home Ground)**")
print(f"✅ Total Matches Played: {total_home_matches}")
print(f"✅ {team_name} Wins: {kkr_home_wins}")
print(f"✅ {unified_opponent_name} Wins: {rcb_home_wins}")

print("\n📌 **Last 5 Head-to-Head Matches with Result Margins:**")
print(top_5_matches)

print("\n🎲 **Total Toss Wins:**")
print(toss_win_count)

print("\n✅ **Correct Toss Decisions (Win Toss & Win Match):**")
print(toss_decision_correct)

print("\n❌ **Wrong Toss Decisions (Win Toss & Lose Match):**")
print(toss_decision_wrong)

print(f"\n🏠 {team_name} Home Toss Wins at {home_venues[0]}: {home_toss_wins}")
print(f"🏆 {team_name} Home Match Wins at {home_venues[0]}: {home_match_wins}")
print(f"📊 {team_name} Home Toss Win %: {home_toss_win_percentage:.2f}%")
print(f"📊 {team_name} Home Match Win %: {home_match_win_percentage:.2f}%")

print(f"\n🚗 {team_name} Away Toss Wins: {away_toss_wins}")
print(f"🏆 {team_name} Away Match Wins: {away_match_wins}")
print(f"📊 {team_name} Away Toss Win %: {away_toss_win_percentage:.2f}%")
print(f"📊 {team_name} Away Match Win %: {away_match_win_percentage:.2f}%")

print("\n📊 **Toss Decision Counts (How many times each team chose Bat or Field):**")
print(toss_decision_counts)

# Printing Toss Decision Results in Required Format
valid_toss_decisions = ['bat', 'field']  # Ensure only valid decisions are used
for toss_winner in toss_decision_results.index.get_level_values(0).unique():
    for toss_decision in valid_toss_decisions:
        if (toss_winner, toss_decision) in toss_decision_results.index:
            team1_wins = toss_decision_results.at[(toss_winner, toss_decision), team_name] if team_name in toss_decision_results.columns else 0
            team2_wins = toss_decision_results.at[(toss_winner, toss_decision), unified_opponent_name] if unified_opponent_name in toss_decision_results.columns else 0

            print(f"\n🎯 {toss_winner} won the toss and chose {toss_decision.capitalize()}")
            print(f"✅ {team1_wins} times {team_name} won the match.")
            print(f"✅ {team2_wins} times {unified_opponent_name} won the match.")



🏏 **Total Head-to-Head Matches Between Kolkata Knight Riders & Royal Challengers Bangalore (RCB):** 34

🏆 **Head-to-Head Wins:**
winner
Kolkata Knight Riders                20
Royal Challengers Bangalore (RCB)    14
Name: count, dtype: int64

🏟 **Head-to-Head at Eden Gardens (KKR Home Ground)**
✅ Total Matches Played: 12
✅ Kolkata Knight Riders Wins: 8
✅ Royal Challengers Bangalore (RCB) Wins: 4

📌 **Last 5 Head-to-Head Matches with Result Margins:**
             date                              team1                              team2                             winner  result_margin
252338 2024-04-21              Kolkata Knight Riders  Royal Challengers Bangalore (RCB)              Kolkata Knight Riders            1.0
246066 2024-03-29  Royal Challengers Bangalore (RCB)              Kolkata Knight Riders              Kolkata Knight Riders            7.0
234512 2023-04-26              Kolkata Knight Riders  Royal Challengers Bangalore (RCB)              Kolkata Knight Riders        

NameError: name 'filtered_comb' is not defined